In [69]:
import numpy as np
import math
import sympy as sym


In [70]:
#전역으로 사용될 변수를 저장해놓는다 
#results 는 output 결과값을 string 으로 저장해놓는 함수로, 나중에 sympy 를 이용해서 깔끔하게 표현한다
results = []
temp_cal = ""

In [81]:
class quantum_computer:
    def __init__(self, n):
        self.n = n
        #크기가 2 ** n 짜리 diagonal matrix 생성 
        self.mat = np.eye(2 ** n, dtype='complex')
         # n이 필요하다면 변수 
        self.a = []
        self.b = []
        # 여기에는 n에 대한 각 변수를 지정해준다 
        for i in range(n): 
           self.a.append('a' + str(i))
           self.b.append('b' + str(i))
           self.a[i] = sym.Symbol(self.a[i])
           self.b[i] = sym.Symbol(self.b[i])
        #이후에는 a[i], b[i]에 대해서만 연산한다면 깔끔하게 표시할 수 있다 

        ind = []
        #string 으로 죄다 저장해서 ind에 넣어놓는 것 -> 이걸 우리의 result 에 넣어놓도록 바꾸어야 한다 
        for i in range(2 ** n):
            tmp = ""
            for j in range(n):
                if j % n != 0: 
                    tmp = ' * ' + tmp
                if (i >> j) % 2 == 1:
                    #tmp = ('(b' + str(n - j) + ')') + tmp
                    tmp = ('b' + str(n-j) + '') + tmp
                else:
                    #tmp = ('(a' + str(n - j) + ')') + tmp
                    tmp = ('a' + str(n-j) + '') + tmp
                
            ind.append(tmp)
        #ind에 각 state에 해당하는 값을 전부 (a1)(a2)..(an) 식으로 붙여서 list 생성 
        self.ind = ind 

       
        
    def result(self, verbose=True):
        result = []
        transformations = sym.parsing.sympy_parser.standard_transformations + (sym.parsing.sympy_parser.function_exponentiation,)
        for i in range(2 ** self.n):
            flag = 0
            #매 줄 시작전에 초기화 시켜줌  
            temp_cal = ""
            if verbose == False:
                print(np.round(self.mat[i], 2))
                #print(self.mat[i])
            else:
                for j in range(2 ** self.n):
                    if self.mat[i][j] != 0:
                        if(flag == 0):
                            flag = 1
                        else:
                            # print(" + ", end='')
                            temp_cal += " + "
                        # print("{0:.2f} {1:}".format(self.mat[i][j], self.ind[j]), end="")
                        temp_cal += "({0:.2f}) * {1:}".format(self.mat[i][j], self.ind[j])
                #print()
                result.append(temp_cal)
                # sym.pprint(sym.sympify(temp_cal, evaluate = True))
                sym.parsing.sympy_parser(temp_cal, transformations = transformations)
        #for index in range(2 ** self.n) :
            #sym.pprint(result[index])
        print()
    
    def output(self, l=0):        
        for elem in self.mat[:, l]:
            print(np.round(elem, 3))
        print()
    
    def marginal(self, l=0):
        print("\t\tProb 0\tProb 1")
        for num in range(self.n):
            print(num, "Qubit:\t", end='')
            count = 0
            for i in range(2 ** self.n):
                if i & (1 << num) == 0:
                    count += self.mat[i, l] * np.conjugate(self.mat[i, l])
            if np.imag(count) != 0:
                print("Wrong")
                return
            count = np.real(count)
            print(np.round(count, 3), "\t", np.round(1-count, 3))
                    
    
    def h(self, a):
        if a >= self.n:
            print("Overflow!")
            return
        
        l = 2 ** a
        t = 0
        while(t < 2 ** self.n):
            for i in range(t, t+l):
                tmp = np.copy(self.mat[i] + self.mat[l + i]) * (0.5 ** 0.5)
                self.mat[l + i] = np.copy(self.mat[i] - self.mat[l + i]) * (0.5 ** 0.5)
                self.mat[i] = tmp
            t += 2 * l
            
    def x(self, a):
        if a >= self.n:
            print("Overflow!")
            return
        
        l = 2 ** a
        t = 0
        while(t < 2 ** self.n):
            for i in range(t, t+l):
                tmp = np.copy(self.mat[l + i])
                self.mat[l + i] = np.copy(self.mat[i])
                self.mat[i] = tmp
            t += 2 * l
            
    def y(self, a):
        if a >= self.n:
            print("Overflow!")
            return
        
        l = 2 ** a
        t = 0
        while(t < 2 ** self.n):
            for i in range(t, t+l):
                tmp = complex(0, -1) * np.copy(self.mat[l + i])
                self.mat[l + i] = complex(0, 1) * np.copy(self.mat[i])
                self.mat[i] = tmp
            t += 2 * l
            
    def z(self, a):
        if a >= self.n:
            print("Overflow!")
            return
        
        l = 2 ** a
        t = 0
        while(t < 2 ** self.n):
            for i in range(t, t+l):
                self.mat[l + i] = -np.copy(self.mat[l + i])
            t += 2 * l
            
    def phase(self, a, theta):
        if a >= self.n:
            print("Overflow!")
            return
        
        l = 2 ** a
        t = 0
        while(t < 2 ** self.n):
            for i in range(t, t+l):
                self.mat[l + i] = np.copy(self.mat[l + i]) * (math.e ** (1j * theta))
            t += 2 * l
    """
    def cnot(self, control, target):
        if control >= self.n or target >= self.n:
            print("Overflow!")
            return
        
        for i in range(2 ** self.n):
            if i & (1 << control) != 0:
                if i & (1 << target) == 0:
                    tmp = np.copy(self.mat[i | (1 << target)])
                    self.mat[i | (1 << target)] = np.copy(self.mat[i])
                    self.mat[i] = tmp
    """
    def cnot(self, control, target):
        if len(control) >= (self.n) or target >= self.n:
            print("Overflow!")
            return
        
        l = []
        for elem in control:
            l.append(1 << elem)
        
        for i in range(2 ** self.n):
            flag = 0
            for elem in l:
                if i & elem == 0:
                    flag = 1
            if flag == 0:
                if i & (1 << target) == 0:
                    tmp = np.copy(self.mat[i | (1 << target)])
                    self.mat[i | (1 << target)] = np.copy(self.mat[i])
                    self.mat[i] = tmp

## IPLEMENT BELOW

In [82]:
# Number of qubits
n = 2
#n=1
qc = quantum_computer(n)

qc.h(0)

qc.x(1)
#qc.cnot([2],1)
# qc.h(2)

#qc.x(0)
#qc.z(0)
#qc.cnot([2],1)

#qc.h(2)
#qc.z(1)
#qc.cnot([1],0)

#qc.h(1)
#qc.cnot([2], 1)
#qc.h(2)
qc.result()

TypeError: 'module' object is not callable

### THIS IS FOR EXPLAINING INTERFACE ###

In [37]:
# Number of qubits
n = 3

qc = quantum_computer(n)

In [38]:
# To print the results of the vector, use result()
qc.result()

1.00+0.00j (a1)(a2)(a3)
1.00+0.00j (a1)(a2)(b3)
1.00+0.00j (a1)(b2)(a3)
1.00+0.00j (a1)(b2)(b3)
1.00+0.00j (b1)(a2)(a3)
1.00+0.00j (b1)(a2)(b3)
1.00+0.00j (b1)(b2)(a3)
1.00+0.00j (b1)(b2)(b3)



In [39]:
# To print the output, use output()
qc.output()

(1+0j)
0j
0j
0j
0j
0j
0j
0j



In [40]:
# To give specific input(only zero or one), give argument the number
qc.output(0b100)

0j
0j
0j
0j
(1+0j)
0j
0j
0j



In [33]:
# To figure out the probability of each qubit, use marginal().
# This takes same argument as method output()
qc.marginal()

qc.marginal(0b011)

		Prob 0	Prob 1
0 Qubit:	1.0 	 -0.0
1 Qubit:	0.5 	 0.5
2 Qubit:	0.5 	 0.5
		Prob 0	Prob 1
0 Qubit:	0.0 	 1.0
1 Qubit:	0.5 	 0.5
2 Qubit:	0.5 	 0.5


In [41]:
# X, Y, Z gate. The qubit 2 is the first qubit.
qc.x(2)
qc.y(1)
qc.z(0)
qc.result()

# Similarly, hadamard gate.
qc.h(2)
qc.result()

0.00-1.00j (b1)(b2)(a3)
-0.00+1.00j (b1)(b2)(b3)
0.00+1.00j (b1)(a2)(a3)
-0.00-1.00j (b1)(a2)(b3)
0.00-1.00j (a1)(b2)(a3)
-0.00+1.00j (a1)(b2)(b3)
0.00+1.00j (a1)(a2)(a3)
-0.00-1.00j (a1)(a2)(b3)

0.00-0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
-0.00+0.71j (a1)(b2)(b3) + -0.00+0.71j (b1)(b2)(b3)
0.00+0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)
0.00-0.71j (a1)(a2)(b3) + 0.00-0.71j (b1)(a2)(b3)
0.00+0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
0.00-0.71j (a1)(b2)(b3) + 0.00+0.71j (b1)(b2)(b3)
0.00-0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)
0.00+0.71j (a1)(a2)(b3) + 0.00-0.71j (b1)(a2)(b3)



In [42]:
# Phase gate.
qc.phase(0, math.pi/2)
qc.result()

0.00-0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
-0.71+0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)
0.00+0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)
0.71-0.00j (a1)(a2)(b3) + 0.71-0.00j (b1)(a2)(b3)
0.00+0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
0.71-0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)
0.00-0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)
-0.71+0.00j (a1)(a2)(b3) + 0.71-0.00j (b1)(a2)(b3)



In [43]:
# CNOT gate is cnot gate with single or multiple control. The first parameter has to be a list.
qc.cnot([1], 0)
qc.result()

qc.cnot([0, 2], 1)
qc.result()

0.00-0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
-0.71+0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)
0.71-0.00j (a1)(a2)(b3) + 0.71-0.00j (b1)(a2)(b3)
0.00+0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)
0.00+0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
0.71-0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)
-0.71+0.00j (a1)(a2)(b3) + 0.71-0.00j (b1)(a2)(b3)
0.00-0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)

0.00-0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
-0.71+0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)
0.71-0.00j (a1)(a2)(b3) + 0.71-0.00j (b1)(a2)(b3)
0.00+0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)
0.00+0.71j (a1)(b2)(a3) + 0.00-0.71j (b1)(b2)(a3)
0.00-0.71j (a1)(a2)(a3) + 0.00+0.71j (b1)(a2)(a3)
-0.71+0.00j (a1)(a2)(b3) + 0.71-0.00j (b1)(a2)(b3)
0.71-0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)



In [44]:
# Final results are...
qc.output()

0j
0j
0j
0.707j
0j
-0.707j
0j
0j



In [45]:
# ex)
qc.y(1)
qc.h(0)
qc.output()

(0.5+0j)
(-0.5+0j)
0j
0j
0j
0j
(0.5+0j)
(-0.5+0j)



In [34]:
# ex 2)
qc = quantum_computer(3)

qc.h(2)
qc.cnot([2], 1)

qc.result()
qc.output()
qc.marginal()

0.71+0.00j (a1)(a2)(a3) + 0.71+0.00j (b1)(a2)(a3)
0.71+0.00j (a1)(a2)(b3) + 0.71+0.00j (b1)(a2)(b3)
0.71+0.00j (a1)(b2)(a3) + 0.71+0.00j (b1)(b2)(a3)
0.71+0.00j (a1)(b2)(b3) + 0.71+0.00j (b1)(b2)(b3)
0.71+0.00j (a1)(b2)(a3) + -0.71+0.00j (b1)(b2)(a3)
0.71+0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)
0.71+0.00j (a1)(a2)(a3) + -0.71+0.00j (b1)(a2)(a3)
0.71+0.00j (a1)(a2)(b3) + -0.71+0.00j (b1)(a2)(b3)

(0.707+0j)
0j
0j
0j
0j
0j
(0.707+0j)
0j

		Prob 0	Prob 1
0 Qubit:	1.0 	 -0.0
1 Qubit:	0.5 	 0.5
2 Qubit:	0.5 	 0.5
